# Survival analysis in credit risk modeling: application of the time-varying ensemble approach
### Svetlana Povolotskaia (591176)


### First packages are imported. After that, all necessary functions, time intervals and hyperparameters are defined. This code is based on the codes provided by Lee, Zame, et al. (2019) via https://github.com/chl8856/SurvivalQuilts.git

In [17]:
import warnings
warnings.filterwarnings("ignore")

In [18]:
import pandas as pd
import numpy as np
import sys, os

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import mean_squared_error

#user defined
from class_SurvivalQuilts import SurvivalQuilts
from utils_eval import calc_metrics

# IMPORT DATASET
### Example for the dataset ndb9 with Y = default 
### To reimplement the experiment, one needs to change the path to the data ('file')

In [19]:
#=================================================================#
##### USER-DEFINED FUNCTIONS
def f_get_Normalization(X, norm_mode):
    num_Patient, num_Feature = np.shape(X)

    if norm_mode == 'standard': #zero mean unit variance
        for j in range(num_Feature):
            if np.std(X[:,j]) != 0:
                X[:,j] = (X[:,j] - np.mean(X[:, j]))/np.std(X[:,j])
            else:
                X[:,j] = (X[:,j] - np.mean(X[:, j]))
    elif norm_mode == 'normal': #min-max normalization
        for j in range(num_Feature):
            X[:,j] = (X[:,j] - np.min(X[:,j]))/(np.max(X[:,j]) - np.min(X[:,j]))
    else:
        print("INPUT MODE ERROR!")

    return X
#=================================================================#



##### DATASET SELECTION
file = '/Users/svpovol/Desktop/Bachelorarbeit/main_ordner/Data/hu_gitHub/ndb9.csv'
SEED    = 1111
data_mode = 'finance'  
        
if data_mode == 'finance':
    df = pd.read_csv(file, sep=',')
    Y           = df[['default']]    # here can be changed to 'payoff'
    T            = df[['time']]
    df = df.drop(['label', 'payoff', 'current_year','default','time'], axis=1)
    X = df
    SEED    = 4321
### option with normalization
#cols = df.columns
#data            = np.asarray(df[cols])
#data            = f_get_Normalization(data, 'standard')
#X = pd.DataFrame(data,columns= cols)

tmp_folder = './results/' + data_mode + ' (seed ' + str(SEED) +')/'
if not os.path.exists(tmp_folder):
    os.makedirs(tmp_folder)


# eval_time_horizons can be selected based on one's interest.
eval_time_horizons = [int(T[Y.iloc[:,0] == 1].quantile(0.25)), int(T[Y.iloc[:,0] == 1].quantile(0.50)), int(T[Y.iloc[:,0] == 1].quantile(0.75))]


tr_X,te_X, tr_T,te_T, tr_Y,te_Y = train_test_split(X, T, Y, test_size=0.2, random_state=1234)
eval_time_horizons, X.shape

([12, 21, 31], (10000, 25))

# TRAIN SURVIVAL QUILTS

In [20]:
print(file)
model_sq = SurvivalQuilts(K=6,num_bo=25, num_cv=5,num_outer=3,step_ahead=2)
model_sq.train(tr_X, tr_T, tr_Y)

#save model
import pickle 
filename = './results/' + 'SurvivalQuilts.sav'
pickle.dump(model_sq, open(filename, 'wb'))

/Users/svpovol/Desktop/Bachelorarbeit/main_ordner/Data/hu_gitHub/ndb9.csv
initial training of underlying models...
CV.. 1/5
CV.. 2/5
CV.. 3/5
CV.. 4/5
CV.. 5/5
TIME K = 0
[[0.         0.         0.         0.         0.         0.99999999]
 [0.         0.         0.         0.         0.         0.99999999]
 [0.         0.         0.         0.         0.         0.99999999]
 [0.         0.         0.         0.         0.         0.99999999]
 [0.         0.         0.         0.         0.         0.99999999]
 [0.         0.         0.         0.         0.         0.99999999]]
beta:  0.4347193579396167
[[0.         0.         0.         0.         0.         0.99999999]]
[[-0.69304046]]
=========== BO Finished ===========
out_itr: 0 | BEST X: [[0.         0.         0.         0.         0.         0.99999999]]
-0.75
out_itr: 0 | Lambda: 1.3467626555042038 | Rho: 0.25
[[0.         0.         0.         0.         0.         0.99999999]]
[[-0.3146599]]
=========== BO Finished ========

# TEST SURVIVAL QUILTS

In [32]:
# Prediction and calibration & discrimination performance for SurvivalQuilt:
pred = model_sq.predict(te_X, eval_time_horizons)
print(' C-Index               Brier Score')
for e_idx, eval_time in enumerate(eval_time_horizons):
    print( calc_metrics(tr_T, tr_Y, te_T, te_Y, pred[:, e_idx], eval_time) )

# Distributin of weights between survival models:
model_sq.quilting_patterns    

 C-Index               Brier Score
(0.9977832086282529, 0.2981981727659572)
(0.9937583129849452, 0.5793767920941533)
(0.9902026867919621, 0.7893484420412433)


array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.99999999],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.99999999],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.99999999],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.99999999],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.99999999],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.99999999]])

In [22]:
# Prediction and calibration & discrimination performance for each survival model from ensemble:
for m in range(model_sq.M):
    tmp_pred_eval = model_sq.underlying_models[m].predict(te_X, eval_time_horizons)
    #Name des Modells:
    model_names = str(model_sq.model_names[m])
    print(model_names) #Name des Modells
    
    if model_sq.quilting_patterns[0][m] > 0.5: #first time check
        #print('For time = ',eval_time,'best weight in this model (',model_sq.quilting_patterns[e_idx][m],')\n')
        tmp_pred_LN=tmp_pred_eval
        name = model_names
    if (model_sq.quilting_patterns[0][m] > 0.) & (model_sq.quilting_patterns[0][m] <0.5):
        print('->',model_sq.quilting_patterns[e_idx][m])

    for e_idx, eval_time in enumerate(eval_time_horizons):
        print( calc_metrics(tr_T, tr_Y, te_T, te_Y, tmp_pred_eval[:, e_idx], eval_time) )
    print('\n')
   

CoxPH
(0.9929917107642888, 0.2996259542553115)
(0.9586020022105567, 0.5869649022522988)
(0.5, 0.22764961401484202)


CoxPHRidge
(0.9920567612430803, 0.2985827479386931)
(0.9590604855505684, 0.5855807653807268)
(0.5, 0.22764961401484202)


Weibull
(0.9903636068006584, 0.29909928684682996)
(0.9586272194817975, 0.5821732081320631)
(0.9728794277731789, 0.796748495275298)


LogNormal
(0.9924807754718719, 0.29906521792034474)
(0.9620912284449539, 0.5817308607187706)
(0.9760172069376557, 0.7977217633559923)


LogLogistic
(0.9931586852744074, 0.29934840361123893)
(0.9614639556928014, 0.5821931897081801)
(0.9763668438936619, 0.7968824155177144)


RandomSurvForest
(0.9979759504166107, 0.29924115181794175)
(0.9945295856541544, 0.5797692494242056)
(0.991301421570603, 0.7876237046672441)




In [27]:
# Compare performance of the best model from ensemble with SurvivalQuilt:
for t in range(len(eval_time_horizons)):
    print('\n',"time: ", eval_time_horizons[t]) 

    print(name,calc_metrics(tr_T, tr_Y, te_T, te_Y, tmp_pred_LN[:, t], eval_time_horizons[t]))
    print("SurvivalQuilt   ",calc_metrics(tr_T, tr_Y, te_T, te_Y, pred[:, t], eval_time_horizons[t]))


 time:  12
RandomSurvForest (0.9979759504166107, 0.29924115181794175)
SurvivalQuilt    (0.9977832086282529, 0.2981981727659572)

 time:  21
RandomSurvForest (0.9945295856541544, 0.5797692494242056)
SurvivalQuilt    (0.9937583129849452, 0.5793767920941533)

 time:  31
RandomSurvForest (0.991301421570603, 0.7876237046672441)
SurvivalQuilt    (0.9902026867919621, 0.7893484420412433)
